# 2.3 自动求梯度
在深度学习中，我们经常需要对函数求梯度（gradient）。本节将介绍如何使用tensorflow2.0提供的GradientTape来自动求梯度。

In [1]:
import tensorflow as tf
print(tf.__version__)
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# tf.test.is_gpu_available( cuda_only=False, min_cuda_compute_capability=None )

2.2.0


2.3.1 简单示例
---

我们先看一个简单例子：对函数 $$y=2x^⊤x$$ 求关于列向量 x 的梯度。我们先创建变量x，并赋初值。

In [2]:
x = tf.reshape(tensor=tf.Variable(range(4),\
                                  dtype=tf.float32),shape=(4,1))

In [3]:
x

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [2.],
       [3.]], dtype=float32)>

函数$$y=2x^⊤x$$ 关于x 的梯度应为$$4x$$ 。现在我们来验证一下求出来的梯度是正确的。

In [4]:
with tf.GradientTape() as t:
    t.watch(x)
    y = 2 * tf.matmul(a=x,b=x,transpose_a=True)
    
dy_dx = t.gradient(target=y,sources=x)
    # tf.GradientTape() 
        # doc: Record operations for automatic differentiatio
        # .watch()
            # doc: Ensures that `tensor` is being traced by this tape
        # .gradient()
            # doc: Computes the gradient using operations recorded in context of this tape.

In [5]:
dy_dx

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[ 0.],
       [ 4.],
       [ 8.],
       [12.]], dtype=float32)>

2.3.2 训练模式和预测模式
---
tf.GradientTape()参数

- persistent: 布尔值，用来指定新创建的gradient tape是否是可持续性的。默认是False，意味着只能够调用一次gradient（）函数。
- watch_accessed_variables: 布尔值，表明这个gradien tap是不是会自动追踪任何能被训练（trainable）的变量。默认是True。要是为False的话，意味着你需要手动去指定你想追踪的那些变量。

In [6]:
with tf.GradientTape(persistent=True) as g:
    g.watch(x)
    y = x * x
    z = y * y
    dz_dx = g.gradient(z, x)  # 108.0 (4*x^3 at x = 3)
    dy_dx = g.gradient(y, x)  # 6.0 (2*x at x =3)
dz_dx,dy_dx

    # 若presistent=False，则将会在第二次使用g.gradient()时报错：
        #RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.

(<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
 array([[  0.],
        [  4.],
        [ 32.],
        [108.]], dtype=float32)>,
 <tf.Tensor: shape=(4, 1), dtype=float32, numpy=
 array([[0.],
        [2.],
        [4.],
        [6.]], dtype=float32)>)

2.3.3 对Python控制流求梯度
---
即使函数的计算图包含了Python的控制流（如条件和循环控制），我们也有可能对变量求梯度。

考虑下面程序，其中包含Python的条件和循环控制。需要强调的是，这里循环（while循环）迭代的次数和条件判断（if语句）的执行都取决于输入a的值。

In [9]:
def f(a):
    b = a * 2
    while tf.norm(b) < 1000:
        b = b * 2
    if tf.reduce_sum(b) > 0:
            # Computes the sum of elements across dimensions of a tensor.
        c = b
    else:
        c = 100 * b
    return c

我们来分析一下上面定义的f函数。事实上，给定任意输入a，其输出必然是 f(a) = x * a的形式，其中标量系数x的值取决于输入a。由于c = f(a)有关a的梯度为x，且值为c / a，我们可以像下面这样验证对本例中控制流求梯度的结果的正确性。

In [8]:
a = tf.random.normal((1,1),dtype=tf.float32)
with tf.GradientTape() as t:
    t.watch(a)
    c = f(a)
t.gradient(c,a) == c/a


<tf.Tensor: shape=(1, 1), dtype=bool, numpy=array([[ True]])>